# Minimal 3-node example of PyPSA linear optimal power flow

Available as a Jupyter notebook at <http://www.pypsa.org/examples/minimal_example_lopf.ipynb>.

In [1]:
import pypsa
import numpy as np

In [2]:
network = pypsa.Network()

In [3]:
#add three buses
for i in range(3):
    network.add("Bus","My bus {}".format(i))

In [4]:
#add three lines in a ring
for i in range(3):
    network.add("Line","My line {}".format(i),
                bus0="My bus {}".format(i),
                bus1="My bus {}".format((i+1)%3),
                x=0.0001,
                s_nom=60)

In [5]:
#add a generator at bus 0
network.add("Generator","My gen 0",
            carrier="gas",
            bus="My bus 0",
            p_nom=100,
            marginal_cost=50)

#add a generator at bus 1
network.add("Generator","My gen 1",
            carrier="gas",
            bus="My bus 1",
            p_nom=100,
            marginal_cost=25)

In [6]:
#add a load at bus 2
network.add("Load","My load",
            bus="My bus 2",
            p_set=100)

In [18]:
network.export_to_netcdf("3bus.nc")

INFO:pypsa.io:Exported network 3bus.nc has buses, lines, loads, generators


<xarray.Dataset>
Dimensions:                   (buses_i: 3, buses_t_marginal_price_i: 1, generators_i: 2, lines_i: 3, loads_i: 1, snapshots: 1)
Coordinates:
  * snapshots                 (snapshots) object 'now'
  * buses_i                   (buses_i) object 'My bus 0' 'My bus 1' 'My bus 2'
  * buses_t_marginal_price_i  (buses_t_marginal_price_i) object 'My bus 0'
  * lines_i                   (lines_i) object 'My line 0' ... 'My line 2'
  * loads_i                   (loads_i) object 'My load'
  * generators_i              (generators_i) object 'My gen 0' 'My gen 1'
Data variables:
    snapshots_weightings      (snapshots) float64 1.0
    buses_t_marginal_price    (snapshots, buses_t_marginal_price_i) float64 50.0
    lines_bus0                (lines_i) object 'My bus 0' 'My bus 1' 'My bus 2'
    lines_bus1                (lines_i) object 'My bus 1' 'My bus 2' 'My bus 0'
    lines_x                   (lines_i) float64 0.0001 0.0001 0.0001
    lines_s_nom               (lines_i) float64

## EnergyModels in PyPSA

We import Julia's EnergyModels and Gurobi from the (local) environment

In [7]:
from julia import Pkg
Pkg.activate(".")
from julia import EnergyModels as EM, Gurobi

.. and are able to mix PyPSA and EnergyModels freely: Data is taken straight out of the pandas dataframes, only short-lived copies during the building phase of each component:

In [8]:
model = EM.EnergyModel(network, optimizer=Gurobi.Optimizer)
EM.build_b(model)

<PyCall.jlwrap Min 50 gas::p[My gen 0,now] + 25 gas::p[My gen 1,now]
Subject to
 gas::p[My gen 0,now] + gas::p[My gen 1,now] = 100.0
 gas::p[My gen 0,now] + gas::p[My gen 1,now] = 100.0
 gas::p[My gen 0,now] + gas::p[My gen 1,now] = 100.0
 -0.0001 lines_AC::p[My line 0,now] - 0.0001 lines_AC::p[My line 1,now] - 0.0001 lines_AC::p[My line 2,now] = 0.0
 lines_AC::p[My line 0,now] ≥ -60.0
 lines_AC::p[My line 1,now] ≥ -60.0
 lines_AC::p[My line 2,now] ≥ -60.0
 gas::p[My gen 0,now] ≥ 0.0
 gas::p[My gen 1,now] ≥ 0.0
 lines_AC::p[My line 0,now] ≤ 60.0
 lines_AC::p[My line 1,now] ≤ 60.0
 lines_AC::p[My line 2,now] ≤ 60.0
 gas::p[My gen 0,now] ≤ 100.0
 gas::p[My gen 1,now] ≤ 100.0
>

... and optimize

In [9]:
EM.optimize_b(model)

... and write back

In [ ]:
EM.store_results_b(model)

In [21]:
#Cheap generator 1 cannot be fully dispatched because of network constraints,
#so expensive generator 0 also has to dispatch
print(network.generators_t.p)

generators_i  My gen 0  My gen 1
snapshots                       
now               20.0      80.0


In [22]:
#network flows
print(network.lines_t.p0)

lines_i    My line 0  My line 1  My line 2
snapshots                                 
now            -20.0       60.0      -40.0


In [23]:
#Line 1 is congested
print(abs(network.lines_t.p0)/network.lines.s_nom)

lines_i    My line 0  My line 1  My line 2
snapshots                                 
now         0.333333        1.0   0.666667


In [24]:
#Power flows towards lower voltage angles
print(network.buses_t.v_ang*180/np.pi)

buses_i    My bus 0  My bus 1  My bus 2
snapshots                              
now             0.0  0.114592 -0.229183


In [25]:
#In linear approximation, all voltage magnitudes are nominal, i.e. 1 per unit
print(network.buses_t.v_mag_pu)

buses_i    My bus 0  My bus 1  My bus 2
snapshots                              
now             1.0       1.0       1.0


In [26]:
#At bus 2 the price is set above any marginal generation costs in the model, because to dispatch to
#it from expensive generator 0, also some dispatch from cheap generator 1 has to be substituted from generator0
#to avoid overloading line 1.
print(network.buses_t.marginal_price)

buses_i    My bus 0  My bus 1  My bus 2
snapshots                              
now            50.0      25.0      75.0


# Alternative `lopf` based on EnergyModels

In [34]:
def lopf_with_energymodels(network, snapshots=None):
    model = EM.EnergyModel(network, optimizer=Gurobi.Optimizer)
    if snapshots is not None:
        EM.set_snapshots_b(snapshots)
    EM.build_b(model)
    EM.optimize_b(model)
    EM.store_results_b(model)

... and go

In [ ]:
lopf_with_energymodels(network)